In [1]:
#Used tools
import numpy as np
import pandas as pd


SyntaxError: invalid character in identifier (<ipython-input-1-9578829bf3b4>, line 5)

In [ ]:
# Implementation of Logistic Regression
class LRegression():
    #Attributes
    _features = None
    _targets = None
    _weights = None
    featt = None
    tarr = None
    
    def __init__(self, feat,target):
        #TARGET IS EXPECTED TO BE changed to 1d array of 1 and zeros
        feat = np.insert(feat,0,1,axis=1) ##Adding x^0
        self._features = feat
        self._weights = np.zeros(self._features[0].size)
        self._targets = target
        self.tarr = self._targets.flatten()
        self.featt = self._features

    def fit(self, tData, corVec, lRate, itera): 
        for x in range(itera):
            N = tData.shape[0]
            grad = np.dot((corVec-self.sig(np.dot(tData,self._weights))),tData) 
            grad *= lRate
            grad /= N
            self._weights = np.add(self._weights,grad)


    def predict(self,tSet):
        return self._classify(self.sig(np.array(np.dot(tSet, self._weights),dtype=np.float32)))
    
    def cost(self,trueV,predV):
        obs = trueV.shape[0]
        
        return ((-trueV*np.log(predV))-((1-trueV)*np.log(1-predV))).sum()/obs
    def sig(self,r):
        return 1.0/(1.0+np.exp(-(np.array(r,dtype=np.float32))))
    
    def printW(self):
        return self._weights
    def _classify(self,pred):
        l = []
        for x in pred:
            if x >= 0.5:
                l.append(1)
            else:
                l.append(0)
        return np.array(l)
        
    def Accu_eval(self,pL,tL):
        n=tL.size
        return np.mean(pL == tL)

In [ ]:
# Implementation of LDA
class LDA():
    _feat = None
    _targ = None
    _N = None
    _p_one = None
    _p_zero = None
    _u_one = None
    _u_zero = None
    _E_one = None
    _E_zero = None
    _E = None
    def __init__(self,features,targets):
        self._feat = features
        self._targ = targets
        self._N = targets.size #Expecting 1d Array of labels
        self._n_one = targets.sum()
        self._n_zero = (self._N-self._targ.sum())
        self._p_one = self._n_one/self._N
        self._p_zero = self._n_zero/self._N
        self._u_zero = 0
        self._u_one = 0
        for x in range(self._N): ##INDICATOR
            if self._targ[x] == 1:
                self._u_one += self._feat[x]
            else:
                self._u_zero += self._feat[x]
        self._u_zero = self._u_zero/self._n_zero #Expecting 1/0 labels passed on
        self._u_one = self._u_one/self._n_one
        self._E_one =0
        self._E_zero =0
        for x in range(self._N):
            if self._targ[x] == 1:
                self._E_one += np.outer((self._feat[x] - self._u_one),(self._feat[x] - self._u_one))/(self._N -2)
            else:
                self._E_zero += np.outer((self._feat[x] - self._u_zero),(self._feat[x] - self._u_zero))/(self._N -2)
        self._E = self._E_one + self._E_zero

    def predict(self,features):
        predResult = []
        a = np.log(self._p_one/self._p_zero)
        b = (np.dot(np.dot(self._u_one.T,np.linalg.inv(np.array(self._E,dtype=np.float32))),self._u_one)/2)
        c = (np.dot(np.dot(self._u_zero.T,np.linalg.inv(np.array(self._E,dtype=np.float32))),self._u_zero)/2)
        for x in range(features.shape[0]):
            d = np.dot(np.dot(features[x].T,np.linalg.inv(np.array(self._E,dtype=np.float32))),(self._u_one-self._u_zero))

            res = a-b+c+d
            if res > 0:
                predResult.append(1)
            else:
                predResult.append(0)
            
        return np.array(predResult)
    
    def Accu_eval(self,pL,tL):
        n=tL.size
        return np.mean(pL == tL)

In [ ]:
#Implementation of 10-fold
#Returns a (kfolds,m,n) mxn is the 2d data array
class k_fold():
    _foldSize = None
    _returnArr = []
    def __init__(self,data,fold=10):
        #Expecting 2d np array
        self._foldSize = np.floor(data.shape[0]/fold)
        np.random.shuffle(data)
        for x in range(fold-1): #Last fold will have an extra element
            self._returnArr.append(data[int(self._foldSize*x):int(self._foldSize*(x+1)),:])
        self._returnArr.append(data[int(self._foldSize*(fold-1)):int(self._foldSize*((fold-1)+1)),:])
        self._returnArr =np.array(self._returnArr)
    def returnFolds(self):
        return self._returnArr

        

In [2]:
def applyToLR(kfold): 
    bestModel = None
    for k in range(10): # Running accuracy on each fold, with the rest combined to apply
        train = None
        train = np.delete(kfold,k,0)
        newT = []
        for x in train:
            for y in x:
                newT.append(y)
        train = np.array(newT)
        test = kfold[k]
        tr_x = train[:,:-1] #Expecting last column to be Y
        tr_y = train[:,-1:].flatten()
        te_x = test[:,:-1] #Expecting last column to be Y
        te_y = test[:,-1:].flatten()
        x = LRegression(tr_x,tr_y)
        x.fit(x.featt,x.tarr,0.3,1000)
        pred = x.predict(np.insert(te_x,0,1,axis=1))
        acc = x.Accu_eval(pred,te_y)
        print("At k="+str(k)+", accurcy = "+str(acc)) 
df = np.array(pd.read_csv('LReg/data/u/sonar.all-data', sep=',',header=None))
x = k_fold(df)
applyToLR(x.returnFolds())

NameError: name 'np' is not defined